In [2]:
import pandas as pd
import numpy as np

In [ ]:
pd.set_option('display.max_columns', None)

In [3]:
# Lectura de datos
df = pd.read_parquet("../Preprocessing/flightsCleaned.parquet")

Queremos predecir los retrasos correspondientes con los 5 aeropuertos que experimentan mayor número de retrasos. En primer lugar debemos seleccionar dichos aeropuertos. Para ello calcularemos el número total de vuelos, el número total de vuelos retrasados y el % de vuelos retrasados para cada uno de los aeropuertos. Una vez hecho esto nos quedaremos con aquellos 5 que muestren un % de retrasos más elevado

In [4]:
# Agrupos los datos de vuelos por aeropuesrtos
airports = pd.DataFrame(df.groupby(df['ORIGIN_AIRPORT'])["FLIGHT_NUMBER"].count())
airports = airports.rename(columns={"FLIGHT_NUMBER":"TOTAL_FLIGHTS"})
airports["DELAYED_FLIGHTS"] =  df[df["ARRIVAL_DELAY"]>0].groupby(df['ORIGIN_AIRPORT'])["FLIGHT_NUMBER"].count()
airports["DELAY_PERCENTAGE"] = np.round(airports['DELAYED_FLIGHTS']/airports['TOTAL_FLIGHTS']*100,2)

# Ordenamos los aeropuertos de tal manera que los 5 con mayor % de retrasos aparezcan los primeros
airports = airports.sort_values('DELAY_PERCENTAGE',ascending=False)
airports = airports.reset_index(level=0, drop=False)
airports.head()

,ORIGIN_AIRPORT,TOTAL_FLIGHTS,DELAYED_FLIGHTS,DELAY_PERCENTAGE
0,GST,76,51,67.11
1,PPG,115,75,65.22
2,ADK,97,61,62.89
3,ILG,95,53,55.79
4,AKN,63,32,50.79


In [4]:
# Seleccionamos el top 5
top_5_delay = airports.loc[0:4]
top_5_delay = list(top_5_delay['ORIGIN_AIRPORT'])
top_5_delay

['GST', 'PPG', 'ADK', 'ILG', 'AKN']

In [5]:
# Filtramos nuestros datos para quedarnos únicamente con los correspondientes a dichos aeropuertos
data_forecast = df[df.ORIGIN_AIRPORT.isin(top_5_delay)]

In [6]:
# Guardamos
data_forecast.to_parquet("dataForecast.parquet", index=False)